Here, we create the function to train and test VGG11 from scratch. We conduct a hyperparameter grid search.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from torchvision.models import vgg11
from sklearn.metrics import precision_score, recall_score

/bin/anaconda3/envs/mohamed-chaaben/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/bin/anaconda3/envs/mohamed-chaaben/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZNK3c107SymBool10guard_boolEPKcl'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
data_path = "/home/mohamed_chaaben/Documents/Colorectal Cancer /"

In [3]:
transform = transforms.Compose([
    transforms.Resize((224, 224)), transforms.ToTensor()])

dataset = datasets.ImageFolder(data_path, transform=transform)

train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_set, val_set, test_set = random_split(dataset, [train_size, val_size, test_size])

In [4]:
model = vgg11(num_classes=3)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 

In [5]:
def train_validate(batch_size, learning_rate):
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_set, batch_size=batch_size)
    test_loader = DataLoader(test_set, batch_size=batch_size)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
    
    best_val_loss = float("inf")
    patience = 5
    no_improvement_count = 0
    
    num_epochs = 50
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch + 1}/{num_epochs} - Loss: {running_loss / len(train_loader)}")

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)

                val_loss += loss.item()

        val_loss /= len(val_loader)

        print(f'Validation Loss: {val_loss}')

        # Check for early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            no_improvement_count = 0
        else:
            no_improvement_count += 1

        if no_improvement_count >= patience:
            print(f'Early stopping after {epoch + 1} epochs with no improvement.')
            break

    print('Training finished.')
    print('Testing begin.')
    model.eval()
    correct = 0
    total = 0
    test_loss = 0.0
    
    all_labels = []
    all_predicted = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)

            loss = criterion(outputs, labels)
            test_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
      
            all_predicted.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    avg_test_loss = test_loss / len(test_loader)
    precision = precision_score(all_labels, all_predicted, average='weighted')
    recall = recall_score(all_labels, all_predicted, average='weighted')
    
    

    print(f'Test Accuracy: {accuracy}%')
    print(f'Test Loss: {avg_test_loss}')
    print('Test Precision', precision)
    print('Test recall', recall)

In [6]:
for lr in [0.01, 0.001, 0.1]:
    for batch_size in [32, 64, 128]:
            print('Settings| Batch size:', batch_size, "Learning rate: ", lr)
            model = vgg11(num_classes=3)
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            model.to(device)
            train_validate(batch_size, lr)

Settings| Batch size: 32 Learning rate:  0.01
Epoch 1/50 - Loss: 1.0991244872411092
Validation Loss: 1.1093965768814087
Epoch 2/50 - Loss: 1.0963530214627584
Validation Loss: 1.098645806312561
Epoch 3/50 - Loss: 1.0991385928789774
Validation Loss: 1.098973035812378
Epoch 4/50 - Loss: 1.093977117538452
Validation Loss: 1.0660111904144287
Epoch 5/50 - Loss: 1.0916889401276906
Validation Loss: 1.0918025970458984
Epoch 6/50 - Loss: 1.099076578617096
Validation Loss: 1.0979984998703003
Epoch 7/50 - Loss: 1.0963033278783163
Validation Loss: 1.095026969909668
Epoch 8/50 - Loss: 1.071766810019811
Validation Loss: 0.8502234220504761
Epoch 9/50 - Loss: 1.0751802400747934
Validation Loss: 1.0506559610366821
Epoch 10/50 - Loss: 1.0781795624891917
Validation Loss: 1.0592236518859863
Epoch 11/50 - Loss: 1.081029699643453
Validation Loss: 1.0995235443115234
Epoch 12/50 - Loss: 1.0921580012639363
Validation Loss: 1.0406447649002075
Epoch 13/50 - Loss: 1.078557276725769
Validation Loss: 1.0734648704528

/bin/anaconda3/envs/mohamed-chaaben/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Test Accuracy: 48.0%
Test Loss: 1.0830287243190564
Test Precision 0.31943216783216777
Test recall 0.48
Settings| Batch size: 64 Learning rate:  0.01
Epoch 1/50 - Loss: 1.0981547387441
Validation Loss: 1.088518738746643
Epoch 2/50 - Loss: 1.0960551373163858
Validation Loss: 1.076623558998108
Epoch 3/50 - Loss: 1.0939310264587403
Validation Loss: 1.0870412588119507
Epoch 4/50 - Loss: 1.085187550385793
Validation Loss: 1.0298875570297241
Epoch 5/50 - Loss: 1.0943962558110556
Validation Loss: 1.1051784753799438
Epoch 6/50 - Loss: 1.0985798009236654
Validation Loss: 1.0956709384918213
Epoch 7/50 - Loss: 1.0974219210942586
Validation Loss: 1.0906521081924438
Epoch 8/50 - Loss: 1.0945538870493572
Validation Loss: 1.0969048738479614
Epoch 9/50 - Loss: 1.0990094216664632
Validation Loss: 1.0910065174102783
Early stopping after 9 epochs with no improvement.
Training finished.
Testing begin.


/bin/anaconda3/envs/mohamed-chaaben/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Test Accuracy: 33.666666666666664%
Test Loss: 1.103078830242157
Test Precision 0.11334444444444443
Test recall 0.33666666666666667
Settings| Batch size: 128 Learning rate:  0.01
Epoch 1/50 - Loss: 1.1002947688102722
Validation Loss: 1.106529951095581
Epoch 2/50 - Loss: 1.0949532797462063
Validation Loss: 1.0960668325424194
Epoch 3/50 - Loss: 1.0886823660448979
Validation Loss: 1.2408233880996704
Epoch 4/50 - Loss: 1.0938011972527755
Validation Loss: 1.057146668434143
Epoch 5/50 - Loss: 1.0940866125257391
Validation Loss: 1.093622088432312
Epoch 6/50 - Loss: 1.1031683055978072
Validation Loss: 1.0974117517471313
Epoch 7/50 - Loss: 1.0995273997909145
Validation Loss: 1.093975305557251
Epoch 8/50 - Loss: 1.0961086844143115
Validation Loss: 1.0948630571365356
Epoch 9/50 - Loss: 1.0936528067839772
Validation Loss: 1.1002867221832275
Early stopping after 9 epochs with no improvement.
Training finished.
Testing begin.


...

/bin/anaconda3/envs/mohamed-chaaben/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Test Accuracy: 36.5%
Test Loss: 1.0977674597188045
Test Precision 0.133225
Test recall 0.365
Settings| Batch size: 64 Learning rate:  0.1
Epoch 1/50 - Loss: 1.10277441183726
Validation Loss: 1.107694387435913
Epoch 2/50 - Loss: 1.1035259389877319
Validation Loss: 1.0904579162597656
Epoch 3/50 - Loss: 1.1019876130421957
Validation Loss: 1.0936261415481567
Epoch 4/50 - Loss: 1.1025376876195272
Validation Loss: 1.0991368293762207
Epoch 5/50 - Loss: 1.10120530128479
Validation Loss: 1.0986888408660889
Epoch 6/50 - Loss: 1.1020641454060873
Validation Loss: 1.100281834602356
Epoch 7/50 - Loss: 1.1039970684051514
Validation Loss: 1.1090750694274902
Early stopping after 7 epochs with no improvement.
Training finished.
Testing begin.


/bin/anaconda3/envs/mohamed-chaaben/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Test Accuracy: 33.666666666666664%
Test Loss: 1.1127476692199707
Test Precision 0.11334444444444443
Test recall 0.33666666666666667
Settings| Batch size: 128 Learning rate:  0.1
Epoch 1/50 - Loss: 1.1004462806802047
Validation Loss: 1.1051403284072876
Epoch 2/50 - Loss: 1.1017726722516512
Validation Loss: 1.1027207374572754
Epoch 3/50 - Loss: 1.1013644086687189
Validation Loss: 1.1016076803207397
Epoch 4/50 - Loss: 1.0994951630893506
Validation Loss: 1.0905174016952515
Epoch 5/50 - Loss: 1.1000991369548596
Validation Loss: 1.0981812477111816
Epoch 6/50 - Loss: 1.1002192591366016
Validation Loss: 1.0994380712509155
Epoch 7/50 - Loss: 1.0996684369287992
Validation Loss: 1.09172523021698
Epoch 8/50 - Loss: 1.1011175168187994
Validation Loss: 1.0968276262283325
Epoch 9/50 - Loss: 1.1004449348700673
Validation Loss: 1.0918461084365845
Early stopping after 9 epochs with no improvement.
Training finished.
Testing begin.
Test Accuracy: 29.833333333333332%
Test Loss: 1.1068106412887573
Test Pre

/bin/anaconda3/envs/mohamed-chaaben/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
